# Introduction to SQL Queries

![sql](img/sql-logo.jpg)

# Objectives

- Describe relational databases
- Connect to a SQLite database and get schema information
- Use SQL SELECT and `pd.read_sql()` to query databases 
- Use WHERE, ORDER BY, and LIMIT to modify queries

# Motivation

Most data aren't stored in static files like CSVs or JSONs. Rather, data are typically stored in **databases** that make it easy for many users to store, update, share, and access data in real time. CSVs and JSONs are just extracts of some data from those databases.

**Structured Query Language (SQL)** is a common language for interacting with databases, and will be invaluable for you in almost any data role. You will use it often to get the data that you need for your analyses.

# Relational Databases

**Relational databases** typically have multiple **tables** containing data, and the tables have defined relationships.

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

## Database Schema

Each database has a **schema** that defines the structure of the database, including the tables and relationships between tables.

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

##  Columns

Similar to how DataFrames can have multiple Series, tables can have multiple **columns** (aka "fields"). Each column has a datatype, but the datatypes available for SQL table columns differ from the datatypes in `pandas`. 

![table example](img/columns.png)

## Keys

A **primary key** uniquely identifies each row in a table. This is often a unique ID number.
A **foreign key** is used in one table to refer to the primary key from another table. 

We **join** tables using these keys to get data from multiple tables at once - we will cover this in a future lesson on SQL Joins.

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

# SQLite

![sqlite](img/SQLite-Python.jpg)

SQLite is one of many tools that exist to create databases. We use it here because it is easy to integrate into a Jupyter Notebook using the `sqlite3` package. 

There are many other database tools out there, and they all work somewhat differently with their own SQL dialects. Just know that the specific methods or syntax you see here will differ slightly in other database implementations.

## _Sidebar: More About SQLite_

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)

## Load a SQLite DB

Import the `sqlite3` package, which will allow us to load a SQLite database.

In [1]:
import sqlite3

In this repository is a `flights.db` file that we can open as a SQLite database. This database contains tables with information about airlines, airports, and flight routes.

In [ ]:
!ls

First, we'll use the `sqlite3` package to create a connection to the database, which is currently just stored in that file on our hard drive.

In [2]:
con = sqlite3.connect('data/flights.db')

Next, we'll create a cursor to interact with the database. Like the cursor for your mouse interacts with pixels on your screen, this cursor will allow us to interact with the elements of the database.

In [3]:
cursor = con.cursor()

## Query the `airports` Table

We will write a simple query using the SQL SELECT statement, which returns data from the database. We write the query as a string, which will then get parsed via the `sqlite3` package.

In this case, we say `SELECT *` to specify that we want data from all columns, and we say `FROM airports` to specify that we want data from the `airports` table.

In [5]:
airports_query = \
"""
SELECT * 
FROM airports
"""

To run this query, we use the `.execute()` method with our cursor.

In [ ]:
cursor.execute(airports_query)

Note that the `.execute()` method didn't actually return our data. The data is now just available in our cursor object. We'll use the `.fetchall()` method to get all the rows from our query.

In [ ]:
cursor.fetchall()

Looks like we got some data, but it's not clear what each element represents. We can view the column names in the cursor's `description` attribute.

In [ ]:
cursor.description

## `pd.read_sql()`

We can get the data and the column names into a nice, tidy DataFrame using `pd.read_sql()`

In [6]:
import pandas as pd

airports_df = pd.read_sql(airports_query, con)
airports_df.head()

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


## Explore the Schema

In SQLite, the schema of our database lives in the `sqlite_master` table. More info [here](https://www.techonthenet.com/sqlite/sys_tables/index.php).

In [ ]:
schema_df = pd.read_sql("""

SELECT *
FROM sqlite_master

""", con)

schema_df

In [ ]:
cursor.execute("""
SELECT *
FROM sqlite_master
""").fetchall()

It looks like there are three tables in our database: airports, airlines, and routes. Each table also has an **index**, which is used to optimize queries for large databases. 

The column names and datatypes for each table are defined in the schema in the `sql` column.

In [ ]:
# Airports table info

print(schema_df['sql'].iloc[0])

## Exercise

Get the columns and datatypes for the airlines table. You can do this with either `pd.read_sql()` or the schema table.

<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

```python 
pd.read_sql("""

SELECT *       
FROM airlines

""", con).info()
```

OR

```python 
print(schema_df['sql'].iloc[2])
```
</details>

In [ ]:
# Your work here
print(schema_df['sql'].iloc[2])


# Writing SQL Queries 

In this section we will build SQL queries using the SELECT statement, showing off a bunch of different clauses and options available. 

## SELECT Statement

SELECT statements can have multiple **clauses**, which must be included in a specific order (more info [here](https://sqlite.org/lang_select.html)). Only SELECT and FROM are required. 

Let's explore the following clauses and structure:

```sql
SELECT columns
FROM table
WHERE condition
ORDER BY columns
LIMIT number
```

## SELECT: Picking Columns

Add the names of the columns that you want after the word `SELECT`, or use `*` to get all columns.

In [ ]:
pd.read_sql("""

SELECT city, country
FROM airports

""", con)

### `DISTINCT`

Use `DISTINCT` to drop duplicates.

In [ ]:
pd.read_sql("""

SELECT DISTINCT city, country
FROM airports

""", con)

### `AS`

Use `AS` to rename columns.

In [ ]:
pd.read_sql("""

SELECT city AS "Airport City", 
       country AS "Airport Country"
FROM airports

""", con)

### Functions

There are dozens of functions that you can use in SELECT statements to modify results - you can see some examples [here](https://sqlite.org/lang_corefunc.html).

In [ ]:
pd.read_sql("""

SELECT UPPER(name) AS "AIRPORT NAMES IN CAPS",
       LENGTH(name) AS "Airport Name Length"
       
FROM airports

""", con)

#### Aggregation

Some functions will aggregate your data and return a table with one row.

In [ ]:
pd.read_sql("""

SELECT COUNT() AS "Number of Airports"
FROM airports

""", con)

#### Datatype Compatibility

Make sure that your column is the right datatype for the function to avoid unexpected results.

In [ ]:
pd.read_sql("""

SELECT name AS "Airport Name", 
       MAX(altitude) AS "Altitude (ft)"
       
FROM airports

""", con)

#### `CAST()`

You could fix this using the `CAST()` function.

In [ ]:
pd.read_sql("""

SELECT name AS "Airport Name", 
       MAX(CAST(altitude as int)) AS "Altitude (ft)"
       
FROM airports

""", con)

### Exercise

Which country has the northern-most airport? 

> _Hint: Look for the highest latitude_

<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

```python 
pd.read_sql("""

SELECT country AS "Country", 
       MAX(CAST(latitude as int)) AS "Latitude"

FROM airports

""", con)
```
</details>

In [ ]:
# Your work here

## FROM: Picking Tables

The `FROM` clause specifies the tables you get data from. You can use aliases here with `AS` - this will be useful for more complex queries involving multiple tables.

In [7]:
pd.read_sql("""

SELECT ap.name AS "Airport Name", 
       MAX(CAST(ap.altitude AS int)) AS "Altitude (ft)"
       
FROM airports AS ap

""", con)

,Airport Name,Altitude (ft)
0,Yading Daocheng,14472


## WHERE: Picking Rows

The `WHERE` clause filters results from your query. This uses conditional logic and operators similar to Python's - you can find more [here](https://sqlite.org/lang_expr.html).

In [8]:
pd.read_sql("""

SELECT name AS "Airport Name", 
       CAST(latitude AS int) AS "Airport Latitude",
       CAST(altitude AS int) AS "Altitude (ft)"
       
FROM airports

WHERE "Altitude (ft)" >= 10000 AND
      "Airport Latitude" BETWEEN 20 AND 50

""", con)

,Airport Name,Airport Latitude,Altitude (ft)
0,Leh,34,10682
1,Hongyuan Airfield,32,11500
2,Lhasa-Gonggar,29,13136
3,Manang,28,11000
4,Jiuzhaigou Huanglong,32,11311
5,Qamdo Bangda Airport,30,14219
6,Maiwa,33,11500
7,Syangboche,27,12309
8,Yushu Batang,32,13000
9,Gunsa,32,13780


### `IS`

The `IS` operator is useful when working with `NULL` values - other operators will not work as expected.  

In [9]:
pd.read_sql("""

SELECT name AS "Airport Name",
       code AS "Airport Code"
       
FROM airports

WHERE "Airport Code" IS NOT NULL

""", con)

,Airport Name,Airport Code
0,Goroka,GKA
1,Madang,MAG
2,Mount Hagen,HGU
3,Nadzab,LAE
4,Port Moresby Jacksons Intl,POM
...,...,...
5875,Mansons Landing Water Aerodrome,YMU
5876,Port McNeill Airport,YMP
5877,Sullivan Bay Water Aerodrome,YTG
5878,Deer Harbor Seaplane,DHB


## ORDER BY: Sorting Results

Use `ORDER BY` to identify the column(s) you want to sort on. Specify `ASC` for ascending order, `DESC` for descending order.

In [10]:
pd.read_sql("""

SELECT name AS "Airport Name", 
       CAST(latitude AS int) AS "Airport Latitude",
       CAST(altitude AS int) AS "Altitude (ft)"
       
FROM airports

WHERE "Altitude (ft)" >= 10000

ORDER BY "Airport Latitude" DESC,
         "Altitude (ft)" DESC 
         
""", con)

,Airport Name,Airport Latitude,Altitude (ft)
0,Summit Camp,72,11000
1,Summit Camp,72,10552
2,Irkutsk-2,52,13411
3,Leh,34,10682
4,Gannan,34,10466
5,Maiwa,33,11500
6,Gunsa,32,13780
7,Yushu Batang,32,13000
8,Hongyuan Airfield,32,11500
9,Jiuzhaigou Huanglong,32,11311


## LIMIT: Number of Results

Specify the maximum number of results you want

In [11]:
pd.read_sql("""

SELECT name AS "Airport Name", 
       CAST(latitude AS int) AS "Airport Latitude",
       CAST(altitude AS int) AS "Altitude (ft)"
       
FROM airports

WHERE "Altitude (ft)" >= 10000

ORDER BY "Airport Latitude" DESC,
         "Altitude (ft)" DESC 
         
LIMIT 10

""", con)

,Airport Name,Airport Latitude,Altitude (ft)
0,Summit Camp,72,11000
1,Summit Camp,72,10552
2,Irkutsk-2,52,13411
3,Leh,34,10682
4,Gannan,34,10466
5,Maiwa,33,11500
6,Gunsa,32,13780
7,Yushu Batang,32,13000
8,Hongyuan Airfield,32,11500
9,Jiuzhaigou Huanglong,32,11311


# Exercises

## Country List

Create a list of countries with airports and order them alphabetically A-Z.

> _Hint: You will need to remove duplicates._

<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

```python 
pd.read_sql("""

SELECT DISTINCT country
FROM airports
ORDER BY country ASC

""", con)
```
</details>

In [ ]:
# Your work here


## Southern Airports

Get the name, country and latitude of the 10 southern-most airports. 

> _Hint: Look for the smallest latitude._

<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

```python 
pd.read_sql("""

SELECT name, country, CAST(latitude AS int) AS "Airport Latitude"
FROM airports
ORDER BY "Airport Latitude" ASC
LIMIT 10

""", con)
``` 
</details>

In [ ]:
# Your work here


## Active UK Airlines

Create a list of active airlines in the United Kingdom from the airlines table. 

> _Hint: You will need to explore the airlines table to figure out how to do this._

<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

```python 
pd.read_sql("""

SELECT name
FROM airlines
WHERE active='Y'
AND country='United Kingdom'

""", con)
```
</details>

In [ ]:
# Your work here


## Explore Routes

Get the column names from the routes table and inspect some raw data. Which columns might be keys that connect this table to the other two tables?

<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

```python 
pd.read_sql("""

SELECT *
FROM routes
LIMIT 5

""", con)
```   
> The airline_id is a foreign key to the airlines table, the source_id and dest_id columns are foreign keys to the airports table
    
</details>

In [ ]:
# Your work here


# Level Up: `CASE`

Use `CASE` to create new columns using conditional logic.

In [ ]:
pd.read_sql("""

SELECT name AS "Airport Name", 
       CAST(altitude AS int) AS "Altitude (ft)",
       CASE 
            WHEN CAST(altitude AS int) > 1000 THEN "High"
            WHEN CAST(altitude AS int) < 100 THEN "Low"
            ELSE "Moderate"
       END AS "Altitude Category"
       
FROM airports
         
LIMIT 20

""", con)

# Level Up: SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

![venn](img/venn.png)

How are these different joins possible?

Notice that I choose a column from each table "on" which to effect the join. This is the means by which I pair up the records from one table with the records of another.

Look back up at the sample diagram under "What is a Relational Database?". We might use the "student_id" column to match up names in the names table with grades in the grades table. But what if there are values in one table's version of "student_id" that don't appear in the other table's version? In that case we need to let the software know whether or not we want to have *all* of the records, regardless of whether they have corresponding entries in all the tables we are joining. This makes for the variety depicted above.

- If I select records from "A INNER JOIN B", then a record will be displayed *only if it exists in both tables*.

- If I select records from "A LEFT JOIN B", then *all relevant records from A will be displayed*, regardless of whether they have representation in B. Records from B with no representation in A will *not* be displayed.

In [ ]:
pd.read_sql("""

    SELECT p.name, l.name, p.country
    FROM airports p
    LEFT JOIN airlines l
    ON p.country=l.country
    ORDER BY l.name
    LIMIT 5
    
""", con)

In [ ]:
pd.read_sql("""
    
    SELECT p.name, l.name, p.country
    FROM airports p
    INNER JOIN airlines l
    ON p.country=l.country
    ORDER BY l.name
    LIMIT 5
    
""", con)